In [ ]:
using JuMP, GLPK
const MOI = JuMP.MathOptInterface

m = 3;
c = zeros(m,m);
#Make diag cost matrix c
for i in 1:m
    for j in 1:m
        if i == j
            c[i,i] = 0;
        end
        if j > i
            c[i,j] = c[j,i] = rand();
        end
        #print(c[i,j], " ")
    end
        #print("\n")
end
#t = 0:m-1;
model = JuMP.Model(JuMP.with_optimizer(GLPK.Optimizer))
@variable(model, x[1:m, 1:m], Bin)
@variable(model, t[1:m, 1:m], Int)
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:m, j in 1:m))
#for i = 1:m
#        @constraint(model, t[i] >= 0)
#end
for i = 1:m, j = 1:m
    if i != j
        @constraint(model,t[j] >= t[i] + 1 - m*(1-x[i,j]))
    end
end
@constraint(model, t[1]==0)
@constraints(model, begin
        # Constraint 1 - Sum over j == 1
        [i in 2:m-1, j in 2:m], sum(x[i,j])==1
        # Constraint 1 - Sum over i == 1
        [i in 2:m, j in 2:m-1], sum(x[i,j])==1
        end)
model
#Solve and test
JuMP.optimize!(model)
    term_status = JuMP.termination_status(model)
    primal_status = JuMP.primal_status(model)
    is_optimal = term_status == MOI.OPTIMAL

    # Check solution
    if is_optimal
        println("Objective is: ", JuMP.objective_value(model))
        println("Optimal solution is:")
        for i in 1:m
            for j in 1:m    
            println("x[$i,$j] = ", JuMP.value(x[i,j]))
            end
        end
    else
        error("No se encontró una solución")
    end